# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data = pd.read_csv("../starter_code/output_data/cities.csv")
city_data_df = pd.DataFrame(city_data)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ustka,54.5805,16.8619,15.92,70,100,11.82,PL,1697259796
1,1,richards bay,-28.7830,32.0377,20.53,64,21,7.08,ZA,1697259797
2,2,albany,42.6001,-73.9662,8.23,87,99,0.45,US,1697259797
3,3,kananga,-5.8958,22.4178,21.59,95,100,1.46,CD,1697259798
4,4,yellowknife,62.4560,-114.3525,11.98,82,0,7.15,CA,1697259798


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.5,)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_low = city_data_df[city_data_df["Max Temp"] > 21]
ideal_temp = ideal_temp_low[ideal_temp_low["Max Temp"] < 28]
no_cloudiness = ideal_temp[ideal_temp["Cloudiness"] == 0]
ideal_cities = no_cloudiness[no_cloudiness["Wind Speed"] < 4.5]
ideal_cities
# Drop any rows with null values
print("Are there null values?")
print(ideal_cities.isnull().any().any())

# Display sample data
ideal_cities

Are there null values?
False


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,saint-pierre,-21.3393,55.4781,25.87,65,0,2.06,RE,1697259806
58,58,tura,25.5198,90.2201,27.54,60,0,1.85,IN,1697259817
80,80,al hada,21.3675,40.2869,22.67,18,0,3.07,SA,1697259824
133,133,taoudenni,22.6783,-3.9836,25.99,18,0,4.20,ML,1697259840
170,170,siwa oasis,29.2041,25.5195,21.21,68,0,2.03,EG,1697259851
212,212,acapulco de juarez,16.8634,-99.8901,27.90,89,0,1.03,MX,1697259864
228,228,xianyang,34.3378,108.7026,23.99,34,0,2.24,CN,1697259868
240,240,bam,29.1060,58.3570,26.71,24,0,1.93,IR,1697259872
265,265,jining,35.4050,116.5814,25.17,27,0,2.61,CN,1697259713
292,292,protaras,35.0125,34.0583,23.33,80,0,0.70,CY,1697259886


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities.copy()
columns_to_drop = ['City_ID', 'Date', 'Cloudiness', 'Max Temp', 'Wind Speed']
hotel_df = hotel_df.drop(columns=columns_to_drop, axis=1)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
23,saint-pierre,-21.3393,55.4781,65,RE,
58,tura,25.5198,90.2201,60,IN,
80,al hada,21.3675,40.2869,18,SA,
133,taoudenni,22.6783,-3.9836,18,ML,
170,siwa oasis,29.2041,25.5195,68,EG,
212,acapulco de juarez,16.8634,-99.8901,89,MX,
228,xianyang,34.3378,108.7026,34,CN,
240,bam,29.1060,58.3570,24,IR,
265,jining,35.4050,116.5814,27,CN,
292,protaras,35.0125,34.0583,80,CY,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":50}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
tura - nearest hotel: No hotel found
al hada - nearest hotel: مريديان الهدا
taoudenni - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
acapulco de juarez - nearest hotel: Hotel del Valle
xianyang - nearest hotel: 如家精选酒店(西安沣西港沣国际店)
bam - nearest hotel: هتل جهانگردی بم
jining - nearest hotel: No hotel found
protaras - nearest hotel: Capo Bay
hilo - nearest hotel: Dolphin Bay Hotel
dalbandin - nearest hotel: SF Al-dawood Restaurant
sakakah - nearest hotel: فندق النزل
del rio - nearest hotel: La Quinta Inn by Wyndham Del Rio
port isabel - nearest hotel: Queen Isabel Inn
les avirons - nearest hotel: Floralys
cuiaba - nearest hotel: Mato Grosso Palace Hotel
ma'rib - nearest hotel: No hotel found


,City,Lat,Lng,Humidity,Country,Hotel Name
23,saint-pierre,-21.3393,55.4781,65,RE,Tropic Hotel
58,tura,25.5198,90.2201,60,IN,No hotel found
80,al hada,21.3675,40.2869,18,SA,مريديان الهدا
133,taoudenni,22.6783,-3.9836,18,ML,No hotel found
170,siwa oasis,29.2041,25.5195,68,EG,فندق الكيلانى
212,acapulco de juarez,16.8634,-99.8901,89,MX,Hotel del Valle
228,xianyang,34.3378,108.7026,34,CN,如家精选酒店(西安沣西港沣国际店)
240,bam,29.1060,58.3570,24,IR,هتل جهانگردی بم
265,jining,35.4050,116.5814,27,CN,No hotel found
292,protaras,35.0125,34.0583,80,CY,Capo Bay


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

hotel_df.isnull().any().any()


# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    color = "City",
    tiles="OSM",
    size=150,
    scale=0.5,
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)